In [14]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [15]:
# abrir a página index (entrar no site da busca jurídica)
import os

caminho = os.getcwd()
arquivo = caminho + r"\index.html"

In [16]:
# importar base de dados
import pandas as pd

tabela = pd.read_excel('Processos.xlsx')
tabela.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Nome      4 non-null      object 
 1   Advogado  4 non-null      object 
 2   Processo  4 non-null      object 
 3   Cidade    4 non-null      object 
 4   Status    0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 292.0+ bytes


In [17]:
from selenium.webdriver import ActionChains
import time

for linha in tabela.index:

    # abrir navegador novo
    navegador.get(arquivo)
    time.sleep(1)

    # abrir lista 
    botao = navegador.find_element(By.XPATH, '/html/body/div/div/button')
    ActionChains(navegador).move_to_element(botao).perform()

    cidade = tabela.loc[linha, 'Cidade']

    # selecionar por pedaço do texto
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

    # trocar de aba
    aba_original = navegador.window_handles[0]
    indice = 1 + linha                          # contador para abas que serão criadas
    aba_selecao = navegador.window_handles[1]

    navegador.switch_to.window(aba_selecao)

    # prencheer formulario com os dados de busca (buscando na tabela)
    navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, 'Nome'])
    navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, 'Advogado'])
    navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, 'Processo'])

    # clicar em pesquisar
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

    # confirmar a pesquisa
    alerta = navegador.switch_to.alert
    alerta.accept()

    # esperando alerta aparecer
    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)

    # pegando texto do alerta
    texto_alerta = alerta.text

    # verificação do alerta
    if "Processo encontrado com sucesso" in texto_alerta:
        alerta.accept()
        tabela.loc[linha, 'Status'] = "Encontrado"
    else:
        tabela.loc[linha, 'Status'] = "Não encontrado"
        alerta.accept()

    navegador.close()     # fecha só a aba atual
    navegador.switch_to.window(navegador.window_handles[0])

C:\Users\Kiko\AppData\Local\Temp\ipykernel_15096\1584236398.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Encontrado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tabela.loc[linha, 'Status'] = "Encontrado"


In [ ]:
navegador.quit()
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Encontrado
3,Carol,Amanda,PC5197,São Paulo,Encontrado
